In [1]:
import pandas as pd
import os

# Import docx
import docx
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Cm
from docx.shared import Pt
from docx.shared import RGBColor

# Import QRCode from pyqrcode 
import pyqrcode 
import png 
from pyqrcode import QRCode

# Import Pillow
from PIL import Image, ImageOps
from PIL import ImageFont
from PIL import ImageDraw 

In [2]:
csv_filename = r"C:\Users\langhe\switchdrive\TIMCI\1 Project details\11 DM\3 ODK\Forms\02 India\1-RCT-LS\01a TIMCI-RCT-Day0 form\facilities.csv"

In [3]:
def set_repeat_table_header(row):
    """ set repeat table row on every new page
    """
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    tblHeader = OxmlElement('w:tblHeader')
    tblHeader.set(qn('w:val'), "true")
    trPr.append(tblHeader)
    return row

def tight_margin(doc):

    current_section = doc.sections[-1]
    current_section.top_margin = Cm(1.4)
    current_section.bottom_margin = Cm(1.4)
    current_section.left_margin = Cm(2)
    current_section.right_margin = Cm(2)
    
def tight_margin2(doc):

    current_section = doc.sections[-1]
    current_section.top_margin = Cm(1.3)
    current_section.bottom_margin = Cm(1.3)
    current_section.left_margin = Cm(2)
    current_section.right_margin = Cm(2)
    
def tight_margin3(doc):

    current_section = doc.sections[-1]
    current_section.top_margin = Cm(1.3)
    current_section.bottom_margin = Cm(1.3)
    current_section.left_margin = Cm(1.3)
    current_section.right_margin = Cm(1.3)

In [4]:
def generate_qr_code(directory, s):
    
    # Generate the QR code 
    url = pyqrcode.create(s)
    
    # Generate image filename
    filename = os.path.join(directory, s+".png")
    
    # Create and save the png file naming "myqr.png" 
    url.png(filename, scale = 3)
    
    img = Image.open(filename)
    rgbimg = img.convert('RGB')
    W, H = rgbimg.size
    draw = ImageDraw.Draw(rgbimg)
    # Add text
    msg = "{}".format(s)
    font = ImageFont.truetype("FONTS/arialbd.ttf", 11)
    w, h = draw.textsize(msg, font=font)
    draw.text(((W+1-w)/2,H-11), msg, font=font, fill="#00c1e4")
    rgbimg.save(filename)

## Template with 4 codes / row (2 for each participant)

In [5]:
def generate_qr_codes_2prow(directory, df, n0, n1, country):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        district = row['lvl2']
        
        word_doc = 'TIMCI {0} - {1} - F{2:04d} {3} - Participants {4} to {5}.docx'.format(country, district, f, fname, n0, n1)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin(doc)
        
        # Add header
        section = doc.sections[0]
        header = section.header
        paragraph = header.paragraphs[0]
        paragraph.text = 'TIMCI {0} - {1} - F{2:04d} {3}'.format(country, district, f, fname)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=int((n1-n0+1)/2)+1, cols=4)
        set_repeat_table_header(tab.rows[0])

        # add the header rows.
        tab.cell(0,0).text = "Sticker for the participant"
        tab.cell(0,0).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,0).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,1).text = "Sticker for the facility research files"
        tab.cell(0,1).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,1).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,2).text = "Sticker for the participant"
        tab.cell(0,2).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,2).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,3).text = "Sticker for the facility research files"
        tab.cell(0,3).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,3).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # add rows
        for p in range(n0-n0+1, int((n1-n0+1)/2)+1):

            # String which represents the QR code 
            s1 = "{0}-F{1:04d}-P{2:04d}".format(country[0], f, n0-1+2*p-1)

            # Generate QR code 
            url = pyqrcode.create(s1)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s1+".png"), scale = 2)
            
            infile1 = os.path.join(os.path.join(directory, s1+".png"))
            
            paragraph = tab.cell(p,0).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,0).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,0).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,1).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,1).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,1).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            # String which represents the QR code 
            s2 = "{0}-F{1:04d}-P{2:04d}".format(country[0], f, n0-1+2*p)

            # Generate QR code 
            url = pyqrcode.create(s2)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s2+".png"), scale = 2)
            
            infile2 = os.path.join(os.path.join(directory, s2+".png"))
            
            paragraph = tab.cell(p,2).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile2)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,2).add_paragraph(s2)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,2).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,3).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile2)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,3).add_paragraph(s2)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,3).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

## List of facilities

In [10]:
import pandas as pd
df = pd.read_csv(csv_filename, sep=",")
df['name'] = df.apply(lambda row: int(row['name'].replace('F','')), axis=1)
df = df[df['intervention'] != 2]
df

,name,label,type,category,ccode,lvl2,intervention
0,1101,Babhani,PHC,rural,I,Deoria,1
1,1102,Baitalpur,PHC,rural,I,Deoria,1
2,1103,Bakhara,PHC,rural,I,Deoria,1
3,1104,Banakata,PHC,rural,I,Deoria,1
5,1005,Baraahaj,CHC,rural,I,Deoria,0
...,...,...,...,...,...,...,...
109,3018,Shuklaganj1,CHC,rural,I,Unnao,0
111,3019,Sikandarpur Sirosi,PHC,rural,I,Unnao,0
112,3020,Sumerpur,PHC,rural,I,Unnao,0
113,3121,Ugu,PHC,rural,I,Unnao,1


## Generate QR codes

In [11]:
generate_qr_codes_2prow(r'QR_codes_India', df, 1, 500, "India")

### Healthcare provider codes

In [ ]:
def generate_hcpqr_codes_2prow(directory, df, n):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI India - F{0:04d} {1} - Providers 1 to {2}.docx'.format(f, fname, n)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=n, cols=2)
        
        # add rows
        for p in range(1, n):

            # String which represents the QR code 
            s1 = "I-F{0:04d}-H{1:02d}".format(f, p)

            # Generate QR code 
            url = pyqrcode.create(s1)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s1+".png"), scale = 2)
            
            infile1 = os.path.join(os.path.join(directory, s1+".png"))
            
            paragraph = tab.cell(p,0).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,0).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,0).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,1).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph = tab.cell(p,1).add_paragraph("First name: ___________________________________")
            paragraph = tab.cell(p,1).add_paragraph("Last name: ____________________________________")
            paragraph = tab.cell(p,1).add_paragraph("Position: _____________________________________")
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

In [ ]:
generate_hcpqr_codes_2prow(r'QR_codes_India', df, 20)